In [4]:
# ─── Standard library modules for filesystem paths, regex parsing, and numerical utilities ───
import re
from pathlib import Path
import numpy as np

# ─── Third-party libraries for PDF parsing and tabular data handling ───
import pdfplumber
import pandas as pd

In [5]:
def extract_all_tables_long(pdf_path: Path) -> pd.DataFrame:
    """
    Open a PDF, extract every table, normalize into a long form DataFrame,
    and return a concatenation of all table records.
    """
    rows = []
    tbl_counter = 0

    with pdfplumber.open(pdf_path) as pdf:
        # iterate over every page
        for page in pdf.pages:
            # extract raw table data per page
            for raw in page.extract_tables():
                tbl_counter += 1
                if not raw or len(raw) < 2:
                    continue  # skip empty or single-row tables

                # find where actual data rows start (first non-blank in column 0)
                start_row = next(
                    (i for i in range(2, len(raw)) if (raw[i][0] or "").strip()),
                    None
                )
                if start_row is None:
                    continue

                # combine all header rows above start_row into flat column names
                header_rows = raw[:start_row]
                num_cols = len(header_rows[0])
                headers = []
                for col in range(num_cols):
                    parts = [
                        (hr[col] or "").replace("\n", " ").strip()
                        for hr in header_rows
                        if (hr[col] or "").strip()
                    ]
                    headers.append(" ".join(parts))

                # build a DataFrame for the data portion
                data_rows = raw[start_row:]
                df = pd.DataFrame(data_rows, columns=headers)

                # detect if there’s a currency column by name
                currency_cols = [c for c in headers if re.search(r'Currency', c, re.IGNORECASE)]
                if currency_cols:
                    cur_col = currency_cols[0]
                    idx_cur = headers.index(cur_col)
                    id_vars = headers[:idx_cur+1]       # include row_label + currency
                    default_currency = None
                else:
                    cur_col = None
                    id_vars = [headers[0]]              # only row_label
                    default_currency = "USD"

                # remaining columns become value columns
                value_vars = [c for c in headers if c not in id_vars]

                # melt into long form with raw values
                long = df.melt(
                    id_vars=id_vars,
                    value_vars=value_vars,
                    var_name="column_header",
                    value_name="v_raw"
                )

                # clean and convert the raw strings into numeric 'value'
                raw_vals = long["v_raw"].astype(str).str.strip()
                is_neg = raw_vals.str.startswith("(") & raw_vals.str.endswith(")")
                base = raw_vals.str.replace(r"[\$\s\(\)%]", "", regex=True)
                is_pct = long["column_header"].str.contains("%", case=False, regex=False)
                clean = pd.Series(index=base.index, dtype="string")
                clean.loc[is_pct]  = base[is_pct].str.replace(",", ".", regex=False)
                clean.loc[~is_pct] = base[~is_pct].str.replace(",",  "", regex=False)
                nums = pd.to_numeric(clean, errors="coerce")
                long["value"] = np.where(is_neg, -nums, nums)
                long.drop(columns="v_raw", inplace=True)

                # rename the first data column to 'row_label'
                long = long.rename(columns={headers[0]: "row_label"})

                # assign currency column or default
                if cur_col:
                    long = long.rename(columns={cur_col: "currency"})
                else:
                    long["currency"] = default_currency

                # add metadata for table identity and page
                long["table_name"]  = f"tbl{tbl_counter}_page{page.page_number}"
                long["page_number"] = page.page_number

                # select and order the final columns
                rows.append(long[[
                    "table_name",
                    "row_label",
                    "column_header",
                    "value",
                    "currency",
                    "page_number"
                ]])

    # concatenate all table outputs into one DataFrame
    return pd.concat(rows, ignore_index=True)

In [6]:
#define your project-root (where you cloned the repo)
PROJECT_ROOT = Path.cwd().parent
# Locate the single Q1-2025 PDF in the bronze folder
BRONZE_Q1     = PROJECT_ROOT / "task1" / "bronze" / "2025_Q1"
PDF_PATH    = list(BRONZE_Q1.glob("*.pdf"))[0]

# Define output Parquet path for the normalized tables
SILVER_DIR  = PROJECT_ROOT / "task2" / "silver"
OUTPUT_PARQ = SILVER_DIR / "q1_2025_tables.parquet"

# Run the extraction function and write results to Parquet
df_all = extract_all_tables_long(PDF_PATH)
df_all.to_parquet(OUTPUT_PARQ, index=False)